# MNIST & Fashion CNN

Before working on the MIL, we'd like to produce a CNN which can properly classify MNIST and Fashion MNIST. Because the fashion MNIST is more difficult, we'll start with it.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.model_selection
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

np.random.seed(42)
tf.random.set_seed(42)

N_CLASSES = 10

print(tf.__version__)

2.4.1


Helper functions to load the datasets.

In [2]:
def preprocess(dataset):
  (x_train, y_train), (x_test, y_test) = dataset

  # Standardize
  x_train = x_train[..., np.newaxis] / 255 #tf.image.per_image_standardization(x_train[..., np.newaxis]).numpy()
  x_test = x_test[..., np.newaxis] / 255 #tf.image.per_image_standardization(x_test[..., np.newaxis]).numpy()

  # One hot
  y_train = tf.one_hot(y_train, N_CLASSES).numpy()
  y_test = tf.one_hot(y_test, N_CLASSES).numpy()

  return (x_train, y_train), (x_test, y_test)

def load_fashion():
  return preprocess(tf.keras.datasets.fashion_mnist.load_data())

def load_mnist():
  return preprocess(tf.keras.datasets.mnist.load_data())

Helper function to train the model.

In [3]:
def evaluate(create_model, load_dataset, n_folds=5):
  (x_train, y_train), (x_test, y_test) = load_dataset()
  print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

  model = create_model()

  return model.fit(
      x_train, y_train,
      callbacks=[ tf.keras.callbacks.EarlyStopping(
          monitor='val_loss',
          patience=5,
          restore_best_weights=True
      ) ],
      validation_data=(x_test, y_test),
      epochs=10,
      batch_size=64
  )

In [4]:
def define_model():
  model = tf.keras.Sequential([
      layers.Input((28, 28, 1)),
      layers.Conv2D(64, 2, activation='relu', padding='same'),
      layers.MaxPool2D(2),
      layers.Conv2D(32, 3, activation='relu'),
      layers.MaxPool2D(2),
      layers.Dropout(0.3),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(N_CLASSES, activation='softmax'),
  ])

  model.compile(
      optimizer=tf.optimizers.Adam(),
      loss=tf.losses.CategoricalCrossentropy(),
      metrics=[
          'accuracy',
          tfa.metrics.F1Score(N_CLASSES, name='f1'),
      ],
  )
  return model

evaluate(define_model, load_fashion)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)
Epoch 1/10
938/938 [==============================] - 33s 34ms/step - loss: 0.7759 - accuracy: 0.7200 - f1: 0.7153 - val_loss: 0.3601 - val_accuracy: 0.8715 - val_f1: 0.8690
Epoch 2/10
938/938 [==============================] - 30s 32ms/step - loss: 0.3757 - accuracy: 0.8622 - f1: 0.8601 - val_loss: 0.3233 - val_accuracy: 0.8792 - val_f1: 0.8779
Epoch 3/10
938/938 [==============================] - 30s 32ms/step - loss: 0.3318 - accuracy: 0.8781 - f1: 0.8778 - val_loss: 0.2884 - val_accuracy: 0.8931 - val_f1: 0.8935
Epoch 4/10
938/938 [==============================] - 30s 32ms/step - loss: 0.2983 - accuracy: 0.8909 - f1: 0.8901 - val_loss: 0.2649 - val_accuracy: 0.9039 - val_f1: 0.9044
Epoch 5/10
938/938 [==============================] - 31s 33ms/step - loss: 0.2829 - accuracy: 0.8968 - f1: 0.8965 - val_loss: 0.2579 - val_accuracy: 0.9069 - val_f1: 0.9046
Epoch 6/10
938/938 [==============================] - 30s 32ms/step 

In [5]:
evaluate(define_model, load_mnist)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)
Epoch 1/10
938/938 [==============================] - 31s 33ms/step - loss: 0.4803 - accuracy: 0.8445 - f1: 0.8421 - val_loss: 0.0542 - val_accuracy: 0.9834 - val_f1: 0.9833
Epoch 2/10
938/938 [==============================] - 30s 32ms/step - loss: 0.0898 - accuracy: 0.9716 - f1: 0.9714 - val_loss: 0.0357 - val_accuracy: 0.9878 - val_f1: 0.9877
Epoch 3/10
938/938 [==============================] - 31s 33ms/step - loss: 0.0681 - accuracy: 0.9784 - f1: 0.9782 - val_loss: 0.0279 - val_accuracy: 0.9906 - val_f1: 0.9905
Epoch 4/10
938/938 [==============================] - 31s 33ms/step - loss: 0.0591 - accuracy: 0.9810 - f1: 0.9809 - val_loss: 0.0282 - val_accuracy: 0.9906 - val_f1: 0.9906
Epoch 5/10
938/938 [==============================] - 31s 33ms/step - loss: 0.0510 - accuracy: 0.9838 - f1: 0.9837 - val_loss: 0.0304 - val_accuracy: 0.9902 - val_f1: 0.9901
Epoch 6/10
938/938 [==============================] - 30s 32ms/step 